In [ ]:
!pip install notebook pandas geopandas shapely duckdb jupysql duckdb-engine

In [ ]:
# import our toolkit
import pandas as pd
import geopandas as gpd
from shapely import wkt
import duckdb 

In [ ]:
# no need to import duckdb_engine, JupySQL will auto-detect driver
# load (or reload) jupysql Jupyter extension to create SQL cells
%reload_ext sql

In [ ]:
# DuckDB in-memory database
%sql duckdb://

In [ ]:
# Run if error
#%sql ROLLBACK

In [ ]:
%sql INSTALL spatial;
%sql LOAD spatial;

In [ ]:
# %%sql
# COPY (
#     SELECT
#         id,
#         level,
#         height,
#         names.primary AS primary_name,
#         sources[1].dataset AS primary_source,
#         geometry
#     FROM read_parquet(
#         's3://overturemaps-us-west-2/release/2025-05-21.0/theme=buildings/type=*/*',
#         hive_partitioning=1
#     )
#     WHERE
#         bbox.xmin > -121.9135
#         AND bbox.xmax < -121.8915
#         AND bbox.ymin > 37.4080
#         AND bbox.ymax < 37.4236
# ) TO 'buildings.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');
%%sql
-- Export San Jose buildings
COPY (
    SELECT
        id,
        level,
        height,
        names.primary AS primary_name,
        sources[1].dataset AS primary_source,
        geometry
    FROM read_parquet(
        's3://overturemaps-us-west-2/release/2025-05-21.0/theme=buildings/type=*/*',
        hive_partitioning=1
    )
    WHERE
        bbox.xmin > -122.10
        AND bbox.xmax < -121.60
        AND bbox.ymin > 37.10
        AND bbox.ymax < 37.45
) TO 'sanjose_buildings.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');

In [ ]:
# %%sql
# COPY(
#     SELECT
#         id,
#         names.primary AS name,
#         CAST(addresses AS JSON) AS addresses,
#         confidence,
#         geometry
#     FROM read_parquet(
#         's3://overturemaps-us-west-2/release/2025-04-23.0/theme=places/type=place/*',
#         filename=true,
#         hive_partitioning=1
#     )
#     WHERE bbox.xmin BETWEEN -121.91 AND -121.87
#       AND bbox.ymin BETWEEN 37.41 AND 37.42
# ) TO 'pois.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');
%%sql
-- Export San Jose POIs
COPY(
    SELECT
        id,
        names.primary AS name,
        CAST(addresses AS JSON) AS addresses,
        confidence,
        geometry
    FROM read_parquet(
        's3://overturemaps-us-west-2/release/2025-04-23.0/theme=places/type=place/*',
        filename=true,
        hive_partitioning=1
    )
    WHERE
        bbox.xmin BETWEEN -122.10 AND -121.60
        AND bbox.ymin BETWEEN 37.10 AND 37.45
) TO 'sanjose_pois.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');

In [ ]:
# Load and project to meters
pois = gpd.read_file("sanjose_pois.geojson").to_crs(epsg=32610)
buildings = gpd.read_file("sanjose_buildings.geojson").to_crs(epsg=32610)

In [ ]:
# Perform spatial join and keep track of which POIs had matches
within = gpd.sjoin(pois, buildings, how="left", predicate="within").reset_index()

# If index_right is not null, the POI was inside a building
poi_inside_flags = within.groupby("index")["index_right"].apply(lambda x: x.notna().any())

# Reindex to pois and assign flag
pois["inside_building"] = pois.index.to_series().map(poi_inside_flags).fillna(False)

outside_pois = pois[~pois["inside_building"]].copy()
buffered = outside_pois.copy()
buffered["geometry"] = buffered.geometry.buffer(50)
candidates = gpd.sjoin(buildings, buffered, how="inner", predicate="intersects")
print(f"{len(candidates)} candidates")

In [ ]:
grouped = candidates.groupby("index_right").agg({
    "id_left": list,
    "primary_name": list,
    "height": list
}).reset_index()

# Rename for readability
grouped = grouped.rename(columns={"id_left": "building_ids"})
for _, row in grouped.iterrows():
    poi_index = row["index_right"]
    poi_name = pois.loc[poi_index, "name"] if "name" in pois.columns else f"POI #{poi_index}"

    print(f"\nPOI {poi_index} ({poi_name}) is near the following buildings:")
    for b_id, b_name, b_height in zip(row["building_ids"], row["primary_name"], row["height"]):
        name_str = f"'{b_name}'" if pd.notna(b_name) else "(no name)"
        print(f"  - ID: {b_id}, Name: {name_str}, Height: {b_height}")

In [ ]:
from shapely.strtree import STRtree

# --- Precompute for optimization ---
# Map of lowercased building names to matching building rows
name_to_buildings = {}
for idx, row in buildings.iterrows():
    name = str(row["primary_name"]).strip().lower()
    if name:
        name_to_buildings.setdefault(name, []).append(row)

# STRtree for fast nearest-neighbor spatial lookup
building_centroids = buildings.geometry.centroid
str_tree = STRtree(building_centroids)
centroid_to_building = dict(zip(building_centroids, buildings.index))

In [ ]:
from shapely.geometry.base import BaseGeometry
def find_best_building_optimized(poi_row):
    poi_geom = poi_row.geometry
    poi_name = str(poi_row["name"]).strip().lower()

    # 1. Name match
    if poi_name in name_to_buildings:
        matches = gpd.GeoDataFrame(name_to_buildings[poi_name])
        matches["dist"] = matches.geometry.centroid.distance(poi_geom)
        return matches.sort_values("dist").iloc[0].geometry.centroid

    # 2. Buffered intersected candidate
    match_candidates = candidates[candidates.index_right == poi_row.name]
    if not match_candidates.empty:
        match_candidates = match_candidates.copy()
        match_candidates["dist"] = match_candidates.geometry.centroid.distance(poi_geom)
        return match_candidates.sort_values("dist").iloc[0].geometry.centroid

    # 3. Nearest building within 200m
    nearby_centroids = str_tree.query(poi_geom.buffer(200))

    # Filter valid geometries
    valid_centroids = [c for c in nearby_centroids if isinstance(c, BaseGeometry)]

    if len(valid_centroids) > 0:
        # Get the nearest centroid
        nearest = min(valid_centroids, key=lambda c: poi_geom.distance(c))

        # Get the matching building geometry
        best_building = buildings.loc[centroid_to_building[nearest]]
        return best_building.geometry.centroid

    # 4. Fallback
    return poi_geom

In [ ]:
# outside_pois["new_geometry"] = outside_pois.apply(find_best_building, axis=1)
outside_pois["new_geometry"] = outside_pois.apply(find_best_building_optimized, axis=1)
inside_pois = pois[pois["inside_building"]].copy()
inside_pois["new_geometry"] = inside_pois.geometry  # keep original
final = pd.concat([inside_pois, outside_pois], ignore_index=True)
final = final.set_geometry("new_geometry")
final = final.set_crs(epsg=32610).to_crs(epsg=4326)
final = final.drop(columns=["geometry", "inside_building"])
final = final.rename(columns={"new_geometry": "geometry"})
final = final.set_geometry("geometry")

In [ ]:
final.to_file("snapped_only_outside_pois.geojson", driver="GeoJSON")

In [ ]:
# Save in Google Drive
from google.colab import drive
drive.mount('/content/drive')
output_path = "/content/drive/MyDrive/snapped_only_outside_pois.geojson"
final.to_file(output_path, driver="GeoJSON")